In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from collections import defaultdict

# NLP
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import emoji

# Viz
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

#Model IndoBERT
import random
import torch
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer

[nltk_data] Downloading package punkt to C:\Users\Oriza
[nltk_data]     Sativa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
df = pd.read_csv('playstore.csv',sep=',')
df = df[["text_data",'sentiment']]

character = ['.',',',';',':','-,','...','?','!','!!!','(',')','[',']','{','}','<','>','"','/','\'','#','-','@',
             'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',
             'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

# hapus karakter yang berulang
def repeatcharClean(text): 
  for i in range(len(character)):
    charac_long = 5
    while charac_long > 2:
      char = character[i]*charac_long 
      text = text.replace(char,character[i])
      charac_long -= 1
  return text

def clean_review(text):
    if pd.notna(text):

        # ubah text menjadi huruf kecil
        text = text.lower()
        # ubah enter menjadi spasi
        text = re.sub(r'\n', ' ', text)
        # hapus emoji
        text = emoji.demojize(text)
        text = re.sub(':[A-Za-z_-]+:', ' ', text) # delete emoji
        # hapus emoticon
        text = re.sub(r"([xX;:]'?[dDpPvVoO3)(])", ' ', text)
        # hapus link
        text = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})", "", text)
        # hapus usename
        text = re.sub(r"@[^\s]+[\s]?", ' ', text)
        # hapus hashtag
        text = re.sub(r'#(\S+)', r'\1', text)
        # hapus angka dan beberapa simbol
        text = re.sub('[^a-zA-Z,.?!]+',' ',text)
        # hapus karakter berulang
        text = repeatcharClean(text)
        # clear spasi
        text = re.sub('[ ]+',' ',text)
    return text

def preprocess_v1(df):
  df_pp = df.copy()
  df_pp.text_data = df_pp.text_data.map(clean_review)  # Corrected the column name

  # Delete empty rows
  df_pp.text_data.replace('', np.nan, inplace=True)
  df_pp.text_data.replace(' ', np.nan, inplace=True)
  df_pp.dropna(subset=['text_data'], inplace=True)
  return df_pp

df_v1 = preprocess_v1(df)
df_v1


,text_data,sentiment
0,"kurang bagus apk ya, masih kurang",0.0
1,"udah mau setaun berlangganan, tapi adanya kena...",0.0
2,sering nge lag.. unrecommended pokoknya.. nyes...,0.0
3,sorry utk bintang nya. karena gak bisa disambu...,0.0
4,duh resolusinya rendah terus dan beberapa kali...,0.0
...,...,...
20815,kurang berjodoh belanja di tokopedia pertama k...,0.0
20816,setelah update terakhir tampilan banyak beruba...,0.0
20817,tampilan versi baru agak gimana gitu. enggak b...,0.0
20818,ini update baru kenapa ya? lagi scroll di kera...,0.0


In [15]:
display(df.iloc[0])

text_data    Kurang bagus apk ya, masih kurang
sentiment                                  0.0
Name: 0, dtype: object